# recreate a coadd

Get all the images that went into a coadd, and then rebuild it.

Maybe tweak some of the image combination parameters for fun, or just rebuild for a few months of observations.

## set up

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

import pandas
pandas.set_option('display.max_rows', 1000)

import lsst.daf.butler as dafButler

import lsst.geom
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

from astropy.wcs import WCS
from astropy import units as u
from astropy.coordinates import SkyCoord

## choose DP0.2 collection

Collection chosen to use same image as in NB 01, the pretty cluster

In [ ]:
repo = 's3://butler-us-central1-panda-dev/dc2/butler-external.yaml'
collection = '2.2i/runs/DP0.2/v23_0_1_rc1/PREOPS-905/pilot_tract4431'

butler = dafButler.Butler(repo, collections=collection)

## choose a coadd

same as we use for NB 01

In [ ]:
my_spherePoint = lsst.geom.SpherePoint(55.745834*lsst.geom.degrees,
                                       -32.269167*lsst.geom.degrees)
print(my_spherePoint)

skymap = butler.get('skyMap')
my_tract = skymap.findTract(my_spherePoint)
my_patch = my_tract.findPatch(my_spherePoint)
my_patch_id = my_tract.getSequentialPatchIndex(my_patch)
print('my_tract = ', my_tract)
print('my_patch = ', my_patch)
print('my_patch_id = ', my_patch_id)

dataId = {'band': 'i', 'tract': 4431, 'patch': 17}
deepCoadd = butler.get('deepCoadd', dataId=dataId)

show the coadd

In [ ]:
# fig = plt.figure(figsize=(6, 4))
# afw_display = afwDisplay.Display(1)
# afw_display.scale('asinh', 'zscale')
# afw_display.mtv(deepCoadd.image)
# plt.gca().axis('off')

get more info about the pixel data (for scaling when using imshow)

In [ ]:
imgdata = np.asarray( deepCoadd.image.array, dtype='float' )
print( np.min(imgdata), np.mean(imgdata), np.max(imgdata) )
del imgdata

get more information about the bounding box and wcs

In [ ]:
deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=dataId)
print( deepCoadd_bbox.beginX, deepCoadd_bbox.beginY, deepCoadd_bbox.endX, deepCoadd_bbox.endY )
print( deepCoadd_bbox.minX, deepCoadd_bbox.minY, deepCoadd_bbox.maxX, deepCoadd_bbox.maxY )
print( deepCoadd_bbox.getCorners )
print(' ')

deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=dataId)
print( deepCoadd_wcs )

deepCoadd_fitsMd = WCS(deepCoadd_wcs.getFitsMetadata())
print( deepCoadd_fitsMd )

del deepCoadd_bbox, deepCoadd_wcs, deepCoadd_fitsMd

## identify coadd inputs

In [ ]:
coadd_inputs = butler.get("deepCoadd_calexp.coaddInputs", dataId)

In [ ]:
coadd_inputs_ccds_table = coadd_inputs.ccds.asAstropy()

In [ ]:
coadd_inputs_ccds_table

In [ ]:
# coadd_inputs_visits_table = coadd_inputs.visits.asAstropy()

In [ ]:
# coadd_inputs_visits_table

## retrieve just one of the calexps which contributed

In [ ]:
use_detector = coadd_inputs_ccds_table['ccd'][0]
use_visit = coadd_inputs_ccds_table['visit'][0]

print('ccd, visit = ', use_detector, use_visit)
calexp = butler.get('calexp', detector=use_detector, visit=use_visit)

In [ ]:
# fig = plt.figure(figsize=(6, 4))
# afw_display = afwDisplay.Display(1)
# afw_display.scale('asinh', 'zscale')
# afw_display.mtv(calexp.image)
# plt.gca().axis('off')

In [ ]:
imgdata = np.asarray( calexp.image.array, dtype='float' )
print( np.min(imgdata), np.mean(imgdata), np.max(imgdata) )
del imgdata

In [ ]:
calexp_bbox = butler.get('calexp.bbox', detector=use_detector, visit=use_visit)
print( calexp_bbox.beginX, calexp_bbox.beginY, calexp_bbox.endX, calexp_bbox.endY )
print( calexp_bbox.minX, calexp_bbox.minY, calexp_bbox.maxX, calexp_bbox.maxY )
print( calexp_bbox.getCorners )
print(' ')

calexp_wcs = butler.get('calexp.wcs', detector=use_detector, visit=use_visit)
print( calexp_wcs )

calexp_fitsMd = WCS(calexp_wcs.getFitsMetadata())
print( calexp_fitsMd )

del calexp_bbox, calexp_wcs, calexp_fitsMd

In [ ]:
del use_detector, use_visit

### do the coadd and the calexp look like they overlap?

`projection = WCS()` DOES NOT WORK FOR COADDS, ORIGIN NOT RECOGNIZED, WORKING ON A FIX

In [ ]:
# fig, ax = plt.subplots(1, 2, figsize=(12, 4), sharex=False, sharey=False)

# plt.subplot(1, 2, 1, projection=WCS(deepCoadd.getWcs().getFitsMetadata()))
# plt.imshow(deepCoadd.image.array, cmap='gray', vmin=0, vmax=2, origin='lower')
# plt.grid(color='white', ls='solid')
# plt.xlabel('Right Ascension')
# plt.ylabel('Declination')

# plt.subplot(1, 2, 2,  projection=WCS(calexp.getWcs().getFitsMetadata()))
# plt.imshow(calexp.image.array, cmap='gray', vmin=-10, vmax=100, origin='lower')
# plt.grid(color='white', ls='solid')
# plt.xlabel('Right Ascension')
# plt.ylabel('Declination')

# plt.show()

In [ ]:
del calexp, deepCoadd

## get corners for the coadd and all input calexps

get corners for the coadd

In [ ]:
deepCoadd_bbox = butler.get('deepCoadd.bbox', dataId=dataId)
deepCoadd_wcs = butler.get('deepCoadd.wcs', dataId=dataId)

X0 = deepCoadd_bbox.beginX
Y0 = deepCoadd_bbox.beginY
X1 = deepCoadd_bbox.endX
Y1 = deepCoadd_bbox.endY
corners = [ deepCoadd_wcs.pixelToSky(X0, Y0), 
            deepCoadd_wcs.pixelToSky(X0, Y1), 
            deepCoadd_wcs.pixelToSky(X1, Y1), 
            deepCoadd_wcs.pixelToSky(X1, Y0) ]
print('corners of the deepCoadd from the wcs:')
print(corners)
print(' ')

corners_deepCoadd = np.zeros( (4,2), dtype='float' )
corners_deepCoadd[0][0] = corners[0].getRa().asDegrees()
corners_deepCoadd[1][0] = corners[1].getRa().asDegrees()
corners_deepCoadd[2][0] = corners[2].getRa().asDegrees()
corners_deepCoadd[3][0] = corners[3].getRa().asDegrees()
corners_deepCoadd[0][1] = corners[0].getDec().asDegrees()
corners_deepCoadd[1][1] = corners[1].getDec().asDegrees()
corners_deepCoadd[2][1] = corners[2].getDec().asDegrees()
corners_deepCoadd[3][1] = corners[3].getDec().asDegrees()
print('corners_deepCoadd as a numpy array:')
print(corners_deepCoadd)

print('corners_deepCoadd in hms dms notation:')
tempra = corners_deepCoadd[:,0]*u.degree
tempdec = corners_deepCoadd[:,1]*u.degree
tempcoords = SkyCoord(ra=tempra, dec=tempdec)
for coord in tempcoords:
    print(coord.to_string('hmsdms'))
del tempra,tempdec,tempcoords

del deepCoadd_bbox, deepCoadd_wcs
del X0,Y0,X1,Y1
del corners

get corners for all the calexps that contributed to our coadd

this takes 10 minutes :(  

save the results to a file and load below

In [ ]:
# t1 = time.time()

# corners_all_ccds = np.zeros( (len(coadd_inputs_ccds_table),4,2), dtype='float' )

# for i, (ccd, visit) in enumerate(zip(coadd_inputs_ccds_table['ccd'], coadd_inputs_ccds_table['visit'])):
#     if i == 10:
#         t2 = time.time()
#         print('time remaining: ', ((t2-t1)/10.0) * (len(coadd_inputs_ccds_table)-10.0) )
    
#     calexp_bbox = butler.get('calexp.bbox', detector=ccd, visit=visit)
#     calexp_wcs = butler.get('calexp.wcs', detector=ccd, visit=visit)

#     X0 = calexp_bbox.beginX
#     Y0 = calexp_bbox.beginY
#     X1 = calexp_bbox.endX
#     Y1 = calexp_bbox.endY
    
#     corner1 = calexp_wcs.pixelToSky(X0, Y0)
#     corner2 = calexp_wcs.pixelToSky(X0, Y1)
#     corner3 = calexp_wcs.pixelToSky(X1, Y1)
#     corner4 = calexp_wcs.pixelToSky(X1, Y0)
    
#     corners_all_ccds[i][0][0] = corner1.getRa().asDegrees()
#     corners_all_ccds[i][1][0] = corner2.getRa().asDegrees()
#     corners_all_ccds[i][2][0] = corner3.getRa().asDegrees()
#     corners_all_ccds[i][3][0] = corner4.getRa().asDegrees()
#     corners_all_ccds[i][0][1] = corner1.getDec().asDegrees()
#     corners_all_ccds[i][1][1] = corner2.getDec().asDegrees()
#     corners_all_ccds[i][2][1] = corner3.getDec().asDegrees()
#     corners_all_ccds[i][3][1] = corner4.getDec().asDegrees()
    
#     del calexp_bbox, calexp_wcs
#     del corner1, corner2, corner3, corner4
#     del X0, X1, Y0, Y1

# t3 = time.time()
# print('time elapsed: ', t3-t1)
# del t1,t2,t3

# with open('corners_all_ccds.dat', 'wb') as f:
#     np.save(f, corners_all_ccds)

In [ ]:
with open('corners_all_ccds.dat', 'rb') as f:
    corners_all_ccds = np.load(f)

In [ ]:
print('corners of the first calexp as a numpy array:')
print(corners_all_ccds[0])

print('corners of the first calexp in hms dms notation:')
tempra = corners_all_ccds[0,:,0]*u.degree
tempdec = corners_all_ccds[0,:,1]*u.degree
tempcoords = SkyCoord(ra=tempra, dec=tempdec)
for coord in tempcoords:
    print(coord.to_string('hmsdms'))
del tempra,tempdec,tempcoords

## plot the bounding boxes

for the coadd and ALL the calexps

In [ ]:
fig = plt.figure(figsize=(6,6))

plt.plot( [corners_deepCoadd[0][0], 
           corners_deepCoadd[1][0], 
           corners_deepCoadd[2][0], 
           corners_deepCoadd[3][0], 
           corners_deepCoadd[0][0]], 
          [corners_deepCoadd[0][1], 
           corners_deepCoadd[1][1], 
           corners_deepCoadd[2][1], 
           corners_deepCoadd[3][1],
           corners_deepCoadd[0][1]],
          lw=4, alpha=0.3, color='grey' )

for i in range(len(coadd_inputs_ccds_table)):    
    plt.plot( [corners_all_ccds[i][0][0], 
               corners_all_ccds[i][1][0], 
               corners_all_ccds[i][2][0], 
               corners_all_ccds[i][3][0], 
               corners_all_ccds[i][0][0]], 
              [corners_all_ccds[i][0][1], 
               corners_all_ccds[i][1][1], 
               corners_all_ccds[i][2][1], 
               corners_all_ccds[i][3][1],
               corners_all_ccds[i][0][1]],
              lw=1, alpha=0.1, color='dodgerblue' )

plt.xlabel( 'ra' )
plt.ylabel( 'dec' )
# plt.xlim([])
# plt.ylim([])
plt.title( 'deepCoadd (grey), input calexps (blue)' )
plt.show()

## identify subset of calexps to combine

e.g., choose a date range of a few months and recreate

In [ ]:
use_detector = coadd_inputs_ccds_table['ccd'][0]
use_visit = coadd_inputs_ccds_table['visit'][0]
calexp_meta = butler.get('calexp.metadata', detector=use_detector, visit=use_visit)
print( calexp_meta['FILTER'], calexp_meta['DATE-END'] )

## recreate the coadd with the calexps

TBD